In [ ]:
# Silver
class CC(): # Chronospatial Computer
    def __init__(self,instructions, memory):
        self.instructions = instructions
        self.memory = memory
    def run_program(self, verbal = False):
        pointer = 0
        self.s_out = ''
        while pointer + 1 < len(self.instructions):
            instruction_code = self.instructions[pointer]
            operand_code = self.instructions[pointer + 1]
            if verbal:
                print(f'Current pointer {pointer}: instruction {instruction_code}, operand {operand_code}, memory {self.memory}')
            jumped = self.process_instruction(instruction_code,operand_code)
            if jumped != None:
                pointer = jumped
            else:
                pointer += 2


    def process_instruction(self,instruction_code, operand_code):
        jumped = None
        if instruction_code == 0: # adv
            val = self.memory[0] >> self.get_operand(operand_code)
            self.memory[0] = val
        elif instruction_code == 1: # bxl
            val = self.memory[1] ^ operand_code
            self.memory[1] = val
        elif instruction_code == 2: # bst
            val = self.get_operand(operand_code) & 0b111 
            self.memory[1] = val
        elif instruction_code == 3: # jnz
            if self.memory[0] != 0:
                jumped = operand_code
        elif instruction_code == 4: # bxc
            val = self.memory[1] ^ self.memory[2] 
            self.memory[1] = val
        elif instruction_code == 5: # out
            val = self.get_operand(operand_code) & 0b111 
            self.s_out += str(val)
        elif instruction_code == 6: # bdv
            val = self.memory[0] >> self.get_operand(operand_code)
            self.memory[1] = val
        elif instruction_code == 7: # bdv
            val = self.memory[0] >> self.get_operand(operand_code)
            self.memory[2] = val
        else:
            return -1

        return jumped

    def get_operand(self,operand_code):
        if operand_code < 4:
            return operand_code
        else:
            return self.memory[operand_code - 4]


instructions = [2,4,1,1,7,5,4,7,1,4,0,3,5,5,3,0]
memory = [30553366, 0, 0]
cc = CC(instructions,memory)
cc.run_program()
cc.s_out

In [ ]:
# Gold
# I hate my life
instructions = [2,4,1,1,7,5,4,7,1,4,0,3,5,5,3,0]
target = ''.join([str(x) for x in instructions])
memory = [0, 0, 0]
input_size_bin = len(instructions)*3
cc = CC(instructions,memory)
output = ''

pointer = input_size_bin - 3
buffer = 1
n = 1 

while pointer >= 0:
    while buffer < 2**(n*3 + 1):
        
        cc.memory = [buffer << pointer, 0, 0]
        cc.run_program()
        if cc.s_out[-n] == target[-n]:
            break
        buffer += 1

    # Backtrack if needed
    for ii in range(1,n):
        if cc.s_out[-ii] != target[-ii]:
            diff = n-ii
            buffer = buffer >> diff * 3
            buffer += 1
            pointer += diff * 3
            n = ii
            break
    # Go forward
    else:
        if pointer == 0:
            break
        buffer = buffer << 3
        pointer -= 3
        n += 1

cc.memory = [buffer, 0, 0]
cc.run_program()
output = cc.s_out
